# Importing the required Libraries


In [2]:
import numpy as np
import time
import copy
import os
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms

OSError: [WinError 1455] The paging file is too small for this operation to complete. Error loading "E:\pythonanaconda\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.

# Data Transformation

In [2]:
transforms = {
    'Train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Importing the dataset

In [3]:
data_dir = 'iip_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), transform=transforms[x])
                  for x in ['Train', 'Val', 'Test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4)
               for x in ['Train', 'Val', 'Test']}
data_size = {x: len(image_datasets[x]) for x in ['Train', 'Val', 'Test']}
class_names = image_datasets['Train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
print(data_size)

{'Train': 2282, 'Val': 255, 'Test': 634}


# Buliding the Model

In [8]:
model_alexnet = models.alexnet(pretrained=True)  #Alex_net 

In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        for phase in ['Train', 'Val']:
            if phase == 'Train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'Train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'Train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'Train':
                scheduler.step()
                
            epoch_loss = running_loss / data_size[phase]
            epoch_acc = running_corrects.double() / data_size[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            if phase == 'Val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model


In [10]:
for param in model_alexnet.parameters():
    param.requires_grad = False


num_ftrs = model_alexnet.classifier[6].in_features
model_alexnet.classifier[6] = nn.Linear(num_ftrs, len(class_names))

In [11]:
model_alexnet = model_alexnet.to(device)                #Enabling GPU

In [12]:
criterion = nn.CrossEntropyLoss()

optimizer_conv = optim.SGD(model_alexnet.classifier[6].parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)
model_alexnet = train_model(model_alexnet, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
Train Loss: 0.8984 Acc: 0.7962
Val Loss: 0.2991 Acc: 0.9255

Epoch 1/24
----------
Train Loss: 0.6152 Acc: 0.8655
Val Loss: 0.7713 Acc: 0.8784

Epoch 2/24
----------
Train Loss: 0.6947 Acc: 0.8615
Val Loss: 0.3032 Acc: 0.9412

Epoch 3/24
----------
Train Loss: 0.5996 Acc: 0.8777
Val Loss: 0.2116 Acc: 0.9686

Epoch 4/24
----------
Train Loss: 0.6399 Acc: 0.8812
Val Loss: 0.3703 Acc: 0.9373

Epoch 5/24
----------
Train Loss: 0.5216 Acc: 0.8948
Val Loss: 0.2221 Acc: 0.9529

Epoch 6/24
----------
Train Loss: 0.5405 Acc: 0.8988
Val Loss: 0.3970 Acc: 0.9451

Epoch 7/24
----------
Train Loss: 0.3680 Acc: 0.9202
Val Loss: 0.1901 Acc: 0.9608

Epoch 8/24
----------
Train Loss: 0.3774 Acc: 0.9150
Val Loss: 0.1143 Acc: 0.9725

Epoch 9/24
----------
Train Loss: 0.3438 Acc: 0.9251
Val Loss: 0.1141 Acc: 0.9765

Epoch 10/24
----------
Train Loss: 0.3437 Acc: 0.9238
Val Loss: 0.1108 Acc: 0.9765

Epoch 11/24
----------
Train Loss: 0.2951 Acc: 0.9246
Val Loss: 0.1008 Acc: 0.9765

Ep

In [7]:
model_resnet = models.resnet18(pretrained=True)   #Res_net

for param in model_resnet.parameters():
    param.requires_grad = False
    
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_ftrs, len(class_names))

model_resnet = model_resnet.to(device)
criterion = nn.CrossEntropyLoss()


optimizer_conv = optim.SGD(model_resnet.fc.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)
model_resnet = train_model(model_resnet, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
Train Loss: 0.6647 Acc: 0.7353
Val Loss: 0.1078 Acc: 0.9804

Epoch 1/24
----------
Train Loss: 0.5716 Acc: 0.8002
Val Loss: 0.0865 Acc: 0.9765

Epoch 2/24
----------
Train Loss: 0.5055 Acc: 0.8116
Val Loss: 0.0771 Acc: 0.9765

Epoch 3/24
----------
Train Loss: 0.4792 Acc: 0.8225
Val Loss: 0.0724 Acc: 0.9843

Epoch 4/24
----------
Train Loss: 0.4724 Acc: 0.8273
Val Loss: 0.0810 Acc: 0.9804

Epoch 5/24
----------
Train Loss: 0.4992 Acc: 0.8234
Val Loss: 0.0618 Acc: 0.9843

Epoch 6/24
----------
Train Loss: 0.4342 Acc: 0.8422
Val Loss: 0.1170 Acc: 0.9529

Epoch 7/24
----------
Train Loss: 0.3865 Acc: 0.8554
Val Loss: 0.0639 Acc: 0.9882

Epoch 8/24
----------
Train Loss: 0.3603 Acc: 0.8699
Val Loss: 0.0554 Acc: 0.9882

Epoch 9/24
----------
Train Loss: 0.3518 Acc: 0.8633
Val Loss: 0.0644 Acc: 0.9882

Epoch 10/24
----------
Train Loss: 0.3542 Acc: 0.8655
Val Loss: 0.0657 Acc: 0.9882

Epoch 11/24
----------
Train Loss: 0.3677 Acc: 0.8668
Val Loss: 0.0501 Acc: 0.9922

Ep

In [13]:
print("Model's state_dict:")
for param_tensor in model_resnet.state_dict():
    print(param_tensor, "\t", model_resnet.state_dict()[param_tensor].size())

Model's state_dict:
conv1.weight 	 torch.Size([64, 3, 7, 7])
bn1.weight 	 torch.Size([64])
bn1.bias 	 torch.Size([64])
bn1.running_mean 	 torch.Size([64])
bn1.running_var 	 torch.Size([64])
bn1.num_batches_tracked 	 torch.Size([])
layer1.0.conv1.weight 	 torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight 	 torch.Size([64])
layer1.0.bn1.bias 	 torch.Size([64])
layer1.0.bn1.running_mean 	 torch.Size([64])
layer1.0.bn1.running_var 	 torch.Size([64])
layer1.0.bn1.num_batches_tracked 	 torch.Size([])
layer1.0.conv2.weight 	 torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight 	 torch.Size([64])
layer1.0.bn2.bias 	 torch.Size([64])
layer1.0.bn2.running_mean 	 torch.Size([64])
layer1.0.bn2.running_var 	 torch.Size([64])
layer1.0.bn2.num_batches_tracked 	 torch.Size([])
layer1.1.conv1.weight 	 torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight 	 torch.Size([64])
layer1.1.bn1.bias 	 torch.Size([64])
layer1.1.bn1.running_mean 	 torch.Size([64])
layer1.1.bn1.running_var 	 torch.Size([64])
layer1.1.bn1.num_batc

In [14]:
model_resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [18]:
PATH = './fixed_resnet18.pth' 
torch.save(model_resnet, PATH)